In [ ]:
import os
import pandas as pd
import folium
import geopandas as gpd

import pulp
from math import sqrt
from shapely.geometry import Point
from shapely.geometry import Polygon
from scipy.spatial.distance import cdist


def moving_population_p_median(gungu, p_num, m_num):
    """
        유동인구 기반 p-Median 문제 풀이 및 시각화
        gungu : 군구명 (예: '수성구')
        p_num : 설치 가능한 후보지 개수
        m_num : 반경(m) 단위 (폴더 구조 맞추기용)
    """

    demand_df = pd.read_csv(
        rf'C:\Users\sh321\OneDrive\바탕 화면\대구 데이콘\모델 동작\수요지점\{gungu}\유동인구\대구광역시_유동인구_{gungu}_5186_pca_가중치추가.csv'
    )
    cand_df = pd.read_csv(
        rf'C:\Users\sh321\OneDrive\바탕 화면\대구 데이콘\모델 동작\입지후보지\{m_num}m\{gungu}_{m_num}m_id추가.csv'
    )

    print(f"[{gungu}] 데이터 불러오기 완료")
    print("수요지점:", demand_df.shape, "후보지점:", cand_df.shape)

    # 거리 행렬 계산
    d_coords = demand_df[['x', 'y']].to_numpy()
    c_coords = cand_df[['x', 'y']].to_numpy()
    dist_matrix_np = cdist(d_coords, c_coords, metric='euclidean')

    # dict 형태로 변환
    dist_matrix = {
        (demand_df.iloc[i].id, cand_df.iloc[j].id): dist_matrix_np[i, j]
        for i in range(len(demand_df))
        for j in range(len(cand_df))
    }

    # p-Median 최적화 모델
    p = p_num
    prob = pulp.LpProblem(f"p_Median_p_{p}", pulp.LpMinimize)

    # 변수 정의
    x = {c: pulp.LpVariable(f"x_{c}", cat="Binary") for c in cand_df['id']}
    y = {(d.id, c.id): pulp.LpVariable(f"y_{d.id}_{c.id}", cat="Binary")
         for _, d in demand_df.iterrows()
         for _, c in cand_df.iterrows()}

    # 목적함수: 총 거리 최소화 (가중치 × 거리 × 할당 여부)
    prob += pulp.lpSum(
        demand_df.loc[demand_df['id'] == d, 'w'].values[0] * dist_matrix[(d, c)] * y[(d, c)]
        for d in demand_df['id'] for c in cand_df['id']
    )

    # 제약조건 1: 각 수요지는 반드시 하나의 후보지에 할당
    for d in demand_df['id']:
        prob += pulp.lpSum(y[(d, c)] for c in cand_df['id']) == 1

    # 제약조건 2: 선택되지 않은 후보지에는 할당 불가
    for d in demand_df['id']:
        for c in cand_df['id']:
            prob += y[(d, c)] <= x[c]

    # 제약조건 3: 설치 가능한 후보지 수 = p
    prob += pulp.lpSum(x[c] for c in cand_df['id']) == p

    # 최적화 수행
    prob.solve(pulp.PULP_CBC_CMD(msg=False))

    # 결과 추출
    selected_cands = [c for c in cand_df['id'] if pulp.value(x[c]) >= 0.5]
    allocation = {
        d: min(
            cand_df['id'],
            key=lambda c: dist_matrix[(d, c)] if pulp.value(y[(d, c)]) >= 0.5 else float("inf")
        )
        for d in demand_df['id']
    }

    # 지도 시각화
    demand_gdf = gpd.GeoDataFrame(
        demand_df, geometry=gpd.points_from_xy(demand_df.x, demand_df.y), crs="EPSG:5186"
    )
    cand_gdf = gpd.GeoDataFrame(
        cand_df, geometry=gpd.points_from_xy(cand_df.x, cand_df.y), crs="EPSG:5186"
    )

    cand_gdf["selected"] = cand_gdf["id"].apply(lambda i: 1 if i in selected_cands else 0)
    demand_gdf["assigned"] = demand_gdf["id"].apply(lambda i: allocation[i] if i in allocation else None)

    demand_gdf4326 = demand_gdf.to_crs(epsg=4326)
    cand_gdf4326 = cand_gdf.to_crs(epsg=4326)

    gdf4326_list = [demand_gdf4326, cand_gdf4326]

    center = [demand_gdf4326.geometry.y.mean(), demand_gdf4326.geometry.x.mean()]
    m = folium.Map(location=center, zoom_start=12)

    # 후보지 (선정된 것만)
    for _, row in cand_gdf4326[cand_gdf4326["selected"] == 1].iterrows():
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=f"선정된 후보지 {row['id']}",
            icon=folium.Icon(color="red", icon="ok-sign")
        ).add_to(m)

    # 수요지 및 할당 관계 표시
    for _, row in demand_gdf4326.iterrows():
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=4,
            color="darkred",
            fill=True,
            popup=f"수요지 {row['id']} → 후보지 {row['assigned']}"
        ).add_to(m)

        # 연결선
        assigned_id = row["assigned"]
        if assigned_id is not None:
            cand_row = cand_gdf4326[cand_gdf4326["id"] == assigned_id].iloc[0]
            folium.PolyLine(
                locations=[[row.geometry.y, row.geometry.x],
                           [cand_row.geometry.y, cand_row.geometry.x]],
                color="red",
                weight=2,
                opacity=0.7
            ).add_to(m)


    m.save(f"./P-Median/유동인구/{gungu}/{m_num}m/P_Median_{gungu}_유동인구_weighted_{m_num}m_p_{p_num}.html")
    print(f"[{gungu}] 지도 저장 완료")

    # 결과 CSV 저장
    result = []
    for i, df in enumerate(gdf4326_list):
        df['WGS4326_x']=df.geometry.x
        df['WGS4326_y']=df.geometry.y
        if i == 0 : cols = ['id', '군구', '행정동', 'x', 'y', 'WGS4326_x','WGS4326_y','assigned']
        else: cols = ['id', 'x', 'y', 'WGS4326_x','WGS4326_y','selected']

        df = df[cols]
        result.append(df)

    gdf4326_list = result

    """
        정제된 gdf4326_list(demand_gdf4326, cand_gdf4326)를 각각 csv 파일로 저장
    """
    filenames = ['수요충족', '선택된후보지']
    filenames = [f'./P-Median/유동인구/{gungu}/{m_num}m/좌표/P_Median_{gungu}_유동인구_weighted_{m_num}m_p_{p_num}_{name}.csv' for name in filenames]

    for df, path in zip(gdf4326_list, filenames):
        df.to_csv(path, index=False)
        print(f"저장 완료: {path}")





In [ ]:
gungu_list = ['남구','북구','서구','중구','달서구','수성구']
p_list = [10,15,20,25]
m = 180

for p in p_list:

    for gungu in gungu_list:
        print(f'p :{p}, 군구 :{gungu}')
        moving_population_p_median(gungu, p,m)

p :10, 군구 :남구
[남구] 데이터 불러오기 완료
수요지점: (170, 8) 후보지점: (553, 4)
[남구] 지도 저장 완료
저장 완료: ./P-Median/유동인구/남구/180m/좌표/P_Median_남구_유동인구_weighted_180m_p_10_수요충족.csv
저장 완료: ./P-Median/유동인구/남구/180m/좌표/P_Median_남구_유동인구_weighted_180m_p_10_선택된후보지.csv
p :10, 군구 :북구
[북구] 데이터 불러오기 완료
수요지점: (604, 8) 후보지점: (2888, 4)
[북구] 지도 저장 완료
저장 완료: ./P-Median/유동인구/북구/180m/좌표/P_Median_북구_유동인구_weighted_180m_p_10_수요충족.csv
저장 완료: ./P-Median/유동인구/북구/180m/좌표/P_Median_북구_유동인구_weighted_180m_p_10_선택된후보지.csv
p :10, 군구 :서구
[서구] 데이터 불러오기 완료
수요지점: (190, 8) 후보지점: (534, 4)
[서구] 지도 저장 완료
저장 완료: ./P-Median/유동인구/서구/180m/좌표/P_Median_서구_유동인구_weighted_180m_p_10_수요충족.csv
저장 완료: ./P-Median/유동인구/서구/180m/좌표/P_Median_서구_유동인구_weighted_180m_p_10_선택된후보지.csv
p :10, 군구 :중구
[중구] 데이터 불러오기 완료
수요지점: (131, 8) 후보지점: (216, 4)
[중구] 지도 저장 완료
저장 완료: ./P-Median/유동인구/중구/180m/좌표/P_Median_중구_유동인구_weighted_180m_p_10_수요충족.csv
저장 완료: ./P-Median/유동인구/중구/180m/좌표/P_Median_중구_유동인구_weighted_180m_p_10_선택된후보지.csv
p :10, 군구 :달서구
[달서구] 데이터 불러오기 완료
수요지점: (574, 8) 후보지점: (1922

In [ ]:
gungu_list = ['남구','북구','서구','중구','달서구','수성구']
p_list = [10,15,20,25]
m = 210

for p in p_list:

    for gungu in gungu_list:
        print(f'p :{p}, 군구 :{gungu}')
        moving_population_p_median(gungu, p,m)

p :10, 군구 :남구
[남구] 데이터 불러오기 완료
수요지점: (170, 8) 후보지점: (401, 4)
[남구] 지도 저장 완료
저장 완료: ./P-Median/유동인구/남구/210m/좌표/P_Median_남구_유동인구_weighted_210m_p_10_수요충족.csv
저장 완료: ./P-Median/유동인구/남구/210m/좌표/P_Median_남구_유동인구_weighted_210m_p_10_선택된후보지.csv
p :10, 군구 :북구
[북구] 데이터 불러오기 완료
수요지점: (604, 8) 후보지점: (2121, 4)
[북구] 지도 저장 완료
저장 완료: ./P-Median/유동인구/북구/210m/좌표/P_Median_북구_유동인구_weighted_210m_p_10_수요충족.csv
저장 완료: ./P-Median/유동인구/북구/210m/좌표/P_Median_북구_유동인구_weighted_210m_p_10_선택된후보지.csv
p :10, 군구 :서구
[서구] 데이터 불러오기 완료
수요지점: (190, 8) 후보지점: (396, 4)
[서구] 지도 저장 완료
저장 완료: ./P-Median/유동인구/서구/210m/좌표/P_Median_서구_유동인구_weighted_210m_p_10_수요충족.csv
저장 완료: ./P-Median/유동인구/서구/210m/좌표/P_Median_서구_유동인구_weighted_210m_p_10_선택된후보지.csv
p :10, 군구 :중구
[중구] 데이터 불러오기 완료
수요지점: (131, 8) 후보지점: (158, 4)
[중구] 지도 저장 완료
저장 완료: ./P-Median/유동인구/중구/210m/좌표/P_Median_중구_유동인구_weighted_210m_p_10_수요충족.csv
저장 완료: ./P-Median/유동인구/중구/210m/좌표/P_Median_중구_유동인구_weighted_210m_p_10_선택된후보지.csv
p :10, 군구 :달서구
[달서구] 데이터 불러오기 완료
수요지점: (574, 8) 후보지점: (1409